In [1]:
import time
import socket
import struct

### Read file of IPs and puts them into a python list

In [2]:
targets = open("targets2.txt")
ip_list = targets.read().splitlines()
print("The following IP addresses will be tested:")

for x in ip_list:
    print(x)
print("\n")

The following IP addresses will be tested:
www.google.com
www.amazon.com




### Creates a UDP and ICMP socket over the default address family

In [3]:
udp = socket.getprotobyname('udp')
udp_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, udp)
print(udp_socket)
#creates a ICMP socket over the default address family
icmp = socket.getprotobyname('icmp')
rcv_socket = socket.socket(socket.AF_INET, socket.SOCK_RAW, icmp)
print(udp_socket)

error: [Errno 1] Operation not permitted

In [4]:
#Constant TTL and port number for each datagram sent out through the UDP socket
ttl = 32
port = 80
timeout = struct.pack("ll", 5, 0)
print(timeout)

               


### Loops through each IP in list

In [5]:
for x in ip_list:
    ip = socket.gethostbyname(x)
    
    ##message of about 1480 bytes of data##
    message="""
    abcdefghijklmnopqrstuvwxyz abcdefghijklmnopqrstuvwxyz abcdefghijklmnopqrstuvwxyz
    """
    
    print("Sending packet to", ip)
    
    #Changes TTL of datagram
    udp_socket.setsockopt(socket.SOL_IP, socket.IP_TTL, ttl)
    
    #Counter of how many datagrams have been sent out(so we don't get stuck if we get an annoying IP)
    tries = 1
    
    #Sets the timeout of the UDP socket to 3 seconds (to avoid an infinite loop)
    #rcv_socket.setsockopt(socket.SOL_SOCKET, socket.SO_RCVTIMEO, timeout)
    #rcv_socket.bind(("", port))
    rcv_socket.settimeout(10.0)
    #Each packet gets sent a maximum of 3 times
    while tries < 4:
        
        #timestamp of when packet is sent
        time_start = time.time()
        #Sends datagram with custom TTL to current IP and constant port number
        udp_socket.sendto(message.encode('utf-8'), (ip,port))
        #udp_socket.bind(('0.0.0.0', 0))
        print("Time start is:" , time_start)
        
        try:
            #read data from receive ICMP socket
            rcv_socket.connect((ip, port))
#             rcv_socket.setblocking(0)#
            print('check!!!!!!!')
            data, address = rcv_socket.recvfrom(4069)
            print(address)
            ttl += 1
            #Timestamp of when packet received (if it is even received)
            time_stop = time.time()
            
            #If the address returned matches the IP
            if (address[0] == x):
                
                #parses the original header to get back the final TTL and protocol
                final_ttl, protocol = struct.unpack("!xxxxxxxxBBxxxxxxxxxx", data[28:48])
                print("The number of hops to this IP is: ",ttl - int(final_ttl), " hops" )
                print("The RTT of the packet is: ", time_stop-time_start," ms")
                
                """
                parses header to get the total length of the datagram. 
                We subtract 28 bytes because 20 represent the new IP header, and 8 represent the ICMP header.
                This leaves us with the 20 byte original header and some of the original payload
                """
                icmp_length = struct.unpack("!xxH", data[0:4])
                print("The number of bytes from the original datagram is:", icmp_length[0] - 28," bytes\n")
                break
                
                
        #deals with the case that no data comes into the receiving socket
        except socket.timeout:
            print("timeout")
            tries = tries + 1
            if tries == 4:
                print("FAILED TO CONNECT. MOVING ON TO NEXT IP\n")
                
udp_socket.close()
rcv_socket.close()

('Sending packet to', '216.58.219.196')


NameError: name 'rcv_socket' is not defined